In [5]:
import json
with open("/media/mydrive/distribution/ann-codes/in-memory/EXPERIMENTS/random/ann_cd2.json") as fin:
    config = json.load(fin)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fvecs import *

dataset = "random"
exp_path = "/media/mydrive/distribution/ann-codes/in-memory/EXPERIMENTS/" + dataset
conf_path = exp_path + "/ann_cd2.json"
l = 50

with open(conf_path) as fin:
    conf = json.load(fin)
x_bins = np.arange(*conf["histogram bins"])[:-1]
distance = fvecs_read(exp_path + "/distances.fvecs")
mre = np.zeros(len(x_bins))
np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [51]:
from read_candidate import load_candidate

def recall_prob(table_probs):
    prod = 1.0
    for p in table_probs:
        prod *= 1 - p
    return 1.0 - prod

def estimate(query, bins):
    candiate_tables = load_candidate(query, conf_path)[:l]
    flat = []
    for ct in candiate_tables:
        flat.extend(ct)
    counters = np.zeros(1000000, dtype=np.int32)
    first_occur = np.full(1000000, -1, dtype=np.int32)
    for table, ct in enumerate(candiate_tables):
        for cand in ct:
            counters[cand] += 1
            if first_occur[cand] == -1:
                first_occur[cand] = table
    candidates = set(flat)

    dist_f = pd.DataFrame({"id": list(range(1000000)), "dist": distance[query, :],
            "count": counters, "table": first_occur})
    distance_group, bins = pd.cut(dist_f["dist"], bins=np.arange(*bins), retbins = True)
    dist_f["dist group"] = distance_group
    cand_f = dist_f.loc[candidates]
    gt_count = dist_f.groupby("dist group").count()
    gt_count = np.array(gt_count["id"])
    cand_f = cand_f.groupby("dist group")
    cand_count = cand_f.count()
    cand_count = np.array(cand_count["id"])
    recall_qh = np.zeros(len(cand_count), dtype=np.float64)
    
    for bin_id, cand_idx in enumerate(cand_f.groups):
        try:
            cand_in_bin = cand_f.get_group(cand_idx)
        except KeyError:
            continue
        recalls = np.zeros(l)
        for table in range(l):
            effective_rows = (cand_in_bin["count"] > 1) | (cand_in_bin["table"] != table)
            cand_effective = set(cand_in_bin[effective_rows]["id"])
            if len(cand_effective) == 0:
                continue
            recalls[table] = len(cand_effective & set(candiate_tables[table])) / len(cand_effective)
        recall_qh[bin_id] = recall_prob(recalls)
    return gt_count, cand_count / recall_qh
    


In [52]:

for qid in range(1000):
    print(qid)
    gt, esti = estimate(qid, conf["histogram bins"])
    esti[np.isnan(esti)] = 0
    esti[np.isinf(esti)] = 0
    plt.clf()
    ax = plt.gca()
    ax.plot(x_bins, gt, label="ground truth")
    ax.plot(x_bins, esti, label="estimated")
    ax.set_xlabel("distance")
    ax.set_ylabel("Number of points")
    ax.legend(loc="upper right")
    plt.savefig("/media/mydrive/distribution/ann-codes/in-memory/EXPERIMENTS/plots/{}_oblivious_{}.pdf".format(
            dataset, qid))
    mre_na = np.abs(gt - esti) / gt
    mre_na[np.isnan(mre_na)] = 0
    mre += mre_na
mre /= 1000
mre

0


UnboundLocalError: local variable 'cand_effective' referenced before assignment

In [49]:
estimate(0, (1 , 1.7, 0.01))

(array([    0,     0,     0,     0,     0,     0,     0,     0,     1,
            0,     0,     1,     4,    10,    20,    28,    66,    86,
          154,   262,   373,   630,  1053,  1529,  2166,  3203,  4419,
         6177,  8451, 11031, 14662, 18565, 23202, 28528, 34129, 39913,
        45858, 51295, 55898, 60014, 61996, 63777, 62905, 60951, 56857,
        52660, 47143, 41002, 33947, 28103, 22278, 17217, 12806,  9202,
         6375,  4267,  2840,  1761,   929,   586,   366,   162,    73,
           48,    16,     5,     0,     0,     0]),
 array([           nan,            nan,            nan,            nan,
                   nan,            nan,            nan,            nan,
        1.00000000e+00,            nan,            nan, 1.00000000e+00,
        4.00046986e+00, 1.01743636e+01, 2.00393069e+01, 2.73161439e+01,
        6.57134219e+01, 8.64648171e+01, 1.51241856e+02, 2.63475219e+02,
        3.73066536e+02, 6.33520132e+02, 1.05988840e+03, 1.52622800e+03,
        2.15096025e